In [1]:
import os
import random

import pandas as pd
import numpy as np

from sklearn.model_selection import KFold

from slim_gsgp.datasets.data_loader import *
from preprocessing import get_X_and_y_tensors
from slim_wrapper import SLIMWrapper
from nested_crossval import run_nested_cv_for_model
from utils import *
import pandas as pd
import plotly.express as px
from scipy.stats import wilcoxon
from itertools import combinations

In [2]:
seed = 1111
np.random.seed(seed)
random.seed(seed)

## 1. Getting X an y tensors

In [3]:
X,y=get_X_and_y_tensors()

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
X

tensor([[ 214.0000, 1725.6000, 1394.0000,  ...,    3.7160,  489.4000,
         2223.3000],
        [ 236.0000, 1769.8000, 1405.4000,  ...,    3.4940,  538.7000,
         2201.8999],
        [ 241.8000, 1724.9000, 1461.7000,  ...,    4.0230,  512.1000,
         2159.8999],
        ...,
        [ 253.1000, 2050.0000, 1669.8000,  ...,    5.1760,  610.0000,
         2648.2000],
        [ 212.1000, 1813.9000, 1468.0000,  ...,    2.8290,  548.4000,
         2262.6001],
        [ 245.7000, 2038.8000, 1655.6000,  ...,    5.9600,  606.9000,
         2704.3000]])

In [6]:
y

tensor([86.1055, 86.1435, 86.4169, 85.9599, 81.6936, 87.5331, 86.8410, 88.2468,
        87.5399, 88.1924, 87.7862, 86.2464, 86.1611, 86.5115, 86.8366, 84.9951,
        86.3812, 88.5102, 85.0129, 85.9784, 88.9662, 84.7507, 87.1134, 87.2657,
        85.5097, 87.1627, 84.8371, 86.8986, 87.4809, 88.2095, 83.4647, 86.3373,
        86.5872, 87.8968, 88.5642, 87.4188, 87.9764, 84.6742, 73.8372, 85.9425,
        86.5554, 87.6478, 87.5996, 86.1728, 88.3539, 87.2642, 86.7197, 85.2699,
        87.7586, 87.8977, 88.1691, 88.4375, 87.1276, 87.5950, 87.9379, 86.5774,
        87.3628, 88.5818, 89.5304, 87.1476, 87.9669, 87.8043, 86.0451, 88.7219,
        88.3917, 89.0096, 85.0285, 87.2786, 87.5549, 87.4504, 88.3307, 89.0777,
        86.8685, 89.4476, 88.6088, 87.6669, 87.6940, 89.0107, 89.6245, 89.4860,
        88.9843, 88.4306, 88.6367, 90.2405, 86.5774, 87.4385, 88.9093, 88.7257,
        88.8421, 88.8315, 87.8210, 88.9991, 88.5073, 90.3756, 91.2113, 88.8250])

## 2. Cross-validation objects

In [7]:
n_outer_cv=10
n_inner_cv=5
outer_cv=KFold(n_splits=n_outer_cv,shuffle=True, random_state=42)

## 3. Applying  SLIM


- `X` will be all the features except the ["CRUDE PROTEIN"], we will try to predict
- `y`will be the ["CRUDE PROTEIN"]
- `fitnesss_function`: rmse
- `minimization`: it will be a minimization problem, we are trying to decrease the rmse

In [8]:
FITNESS_FUNCTION = 'rmse'
MINIMIZATION = True

__`Search space definition`__

- `initializer`: how new random trees are initialized. See [`slim_gsgp` initializers](https://github.com/DALabNOVA/slim/blob/main/slim_gsgp/initializers/initializers.py);
- `tree_constants`: the constants to be used in the terminal set;
- `tree_functions`: the function set (tree internal nodes);
- `prob_const`: the probability for choosing constants instead of dataset features on tree terminals;
- `init_depth`: max depth for tree initialisation;


The following hyperparameter options are the same as for **GP and GSGP**:

- `pop_size`: the size of the population of candidate solutions.
- `elitism`: should the elite(s) be preserved at each generation?
- `n_elits`: if using elitism, how many solutions should be kept?
- Selection method. Only tournament selection in available on `slim_gsgp` libraya, as this is the most commonly used. It requires the definition of the `tournament_size` hyperparameter: how many solutions should participate in the tournament of tournament selection?

The following hyperparameter options are the same as only for **GSGP**:

- `ms_lower`: lower bound for generating the random number used as mutation step.
- `ms_upper`: upper bound for generating the random number used as mutation step.
- `reconstruct`: whether to store the structure of individuals.

**Additionally, SLIM requires:**
- `slim_version`: So far, mutator variation (SLIM+SIG2, SLIM+SIG1, SLIM+ABS, SLIM*SIG2, SLIM*SIG1, SLIM*ABS)
- `p_inflate`: The probability of applying inflate mutation instead of deflate mutation.
- `copy_parent`: Whether to copy the original parent when mutation is impossible (due to depth rules or mutation constraints)

In [9]:
INITIALIZER = 'rhh'
TREE_CONSTANTS = [random.uniform(0, 1) for _ in range(9)]+[ -1.]
TREE_FUNCTIONS = ['add', 'subtract', 'multiply', 'divide']
PROB_CONSTANT = 0.9
MAX_INIT_DEPTH = 4
POP_SIZE = 200
P_XO = 0.9
ELISTISM = True
N_ELITES = 2
TOURNAMENT_SIZE = int(POP_SIZE*0.1)
print(f'TOURNAMENT_SIZE: {TOURNAMENT_SIZE}')

MS_LOWER = 0
MS_UPPER = 1
RECONSTRUCT = True
GENERATIONS = 20
VERBOSE = 1
DATASET_NAME="sustavianfeed"
LOG_LEVEL = 2

SLIM_VERSION = 'SLIM*SIG2'
P_INFLATE = 0.3
COPY_PARENT = False

TOURNAMENT_SIZE: 20


__`Solve settings`__

Same as available for GP and GSGP:

## 4. Hyperparameter tunning with Nested Crossfold validation

We chose nested cross-validation instead of a random train-test split because it gives a fairer and more reliable way to check our model’s performance. It has two steps: one for finding the best model settings (hyperparameters) and another for testing how well the model works on new data. This is important because if we tune and test the model on the same data, the results might look better than they actually are. 
By keeping these steps separate, nested cross-validation helps avoid this problem and gives a more honest estimate of how our model will perform on new, unseen data.

- For the inner fold of the nested cross-validation, we applied Grid Search to take advantage of all its built-in features like systematic parameter tuning and parallel processing. To make this work with the SLIM library, we implemented a custom wrapper called SLIMWrapper, which follows the BaseEstimator interface. This allowed us to integrate the SLIM seamlessly into the grid search process, ensuring efficient and consistent hyperparameter tuning within the inner loop.

In [10]:
estimator=SLIMWrapper(
    init_depth=MAX_INIT_DEPTH,
    tree_constants=TREE_CONSTANTS,
    tree_functions=TREE_FUNCTIONS,
    prob_const = PROB_CONSTANT,
    dataset_name=DATASET_NAME,
    fitness_function=FITNESS_FUNCTION,
    minimization=MINIMIZATION,
    # --
    # SLIM instance 
    pop_size=POP_SIZE,
    initializer=INITIALIZER,
    ms_lower = MS_LOWER,
    ms_upper = MS_UPPER,
    reconstruct = RECONSTRUCT,
    p_inflate=P_INFLATE,
    copy_parent=COPY_PARENT,
    # ---
    # Solve settings
    n_iter=GENERATIONS,
    elitism=ELISTISM,
    n_elites=N_ELITES,
    test_elite=True,
    log_level=LOG_LEVEL,
    verbose=0,
    n_jobs=1,
    seed=seed
)

For this model we shall focus our analysis in 'slim_version'. Our goal is to find which version allows us to have the best performance with all other parameters fixed, for fair comparison, while also trying to make sure our model doesn't grow excessively

In [11]:
param_grid={
    'slim_version': ['SLIM+SIG2', 'SLIM+SIG1', 'SLIM+ABS', 'SLIM*SIG2', 'SLIM*SIG1', 'SLIM*ABS'],
}

In [12]:
summary, general_summary, all_grid_results=run_nested_cv_for_model(X,y, estimator, param_grid, outer_cv, n_inner_cv, model_name="SLIM")


 Outer fold 1


c:\Users\elton\AppData\Local\Programs\Python\Python310\lib\site-packages\slim_gsgp\main_slim.py:161: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")



Grid search completed!
Best parameters: {'slim_version': 'SLIM*ABS'}
Best score: -6.58449342250824
Training with the best hyparameters of 1 fold
Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     sustavianfeed       |       0      |   15.209765434265137  |   14.483480453491211     |   0.046767473220825195 |      3           |
|     sustavianfeed       |       1      |   15.209765434265137  |   14.483480453491211     |   0.07995414733886719  |      3           |
|     sustavianfeed       |       2      |   15.209765434265137  |   14.483480453491211     |   0.05091381072998047  |      3           |
|     sus

c:\Users\elton\AppData\Local\Programs\Python\Python310\lib\site-packages\slim_gsgp\main_slim.py:161: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")



Grid search completed!
Best parameters: {'slim_version': 'SLIM*SIG1'}
Best score: -4.766940665245056
Training with the best hyparameters of 2 fold
Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     sustavianfeed       |       0      |   15.64592456817627   |   9.697028160095215      |   0.0426487922668457   |      3           |
|     sustavianfeed       |       1      |   15.64592456817627   |   9.697028160095215      |   0.06966018676757812  |      3           |
|     sustavianfeed       |       2      |   15.64592456817627   |   9.697028160095215      |   0.04104113578796387  |      3           |
|     s

...


Grid search completed!
Best parameters: {'slim_version': 'SLIM*SIG1'}
Best score: -4.033915281295776
Training with the best hyparameters of 8 fold
Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     sustavianfeed       |       0      |   15.426959991455078  |   11.960426330566406     |   0.052324771881103516 |      3           |
|     sustavianfeed       |       1      |   15.426959991455078  |   11.960426330566406     |   0.0941324234008789   |      3           |
|     sustavianfeed       |       2      |   15.426959991455078  |   11.960426330566406     |   0.06562924385070801  |      3           |
|     s

c:\Users\elton\AppData\Local\Programs\Python\Python310\lib\site-packages\slim_gsgp\main_slim.py:161: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")



Grid search completed!
Best parameters: {'slim_version': 'SLIM*SIG1'}
Best score: -7.47721631526947
Training with the best hyparameters of 9 fold
Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     sustavianfeed       |       0      |   15.693072319030762  |   7.937129497528076      |   0.17485642433166504  |      3           |
|     sustavianfeed       |       1      |   15.693072319030762  |   7.937129497528076      |   0.12696075439453125  |      3           |
|     sustavianfeed       |       2      |   15.693072319030762  |   7.937129497528076      |   0.056778907775878906 |      3           |
|     su

c:\Users\elton\AppData\Local\Programs\Python\Python310\lib\site-packages\slim_gsgp\main_slim.py:161: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")



Grid search completed!
Best parameters: {'slim_version': 'SLIM*SIG1'}
Best score: -4.982069802284241
Training with the best hyparameters of 10 fold
Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     sustavianfeed       |       0      |   15.514117240905762  |   10.815879821777344     |   0.05015826225280762  |      3           |
|     sustavianfeed       |       1      |   15.514117240905762  |   10.815879821777344     |   0.10465288162231445  |      3           |
|     sustavianfeed       |       2      |   15.514117240905762  |   10.815879821777344     |   0.06286048889160156  |      3           |
|     

In [13]:
# Log level 2
# -----------
# 0  - Algorithm
# 1  - Instance ID
# 2  - Dataset
# 3  - Seed
# 4  - Generation
# 5  - Fitness
# 6  - Running time
# 7  - Population nodes
# 8  - Test fitness
# 9  - Elite nodes
# 10 - Genotype diversity: gsgp_pop_div_from_vectors (Calculate the diversity of a population from semantic vectors)
# 11 - Phenotype diversity: sd(pop.fit)
# 12 - Log level

fold_1_result=pd.read_csv("./results/SLIM/logs_best_params_fold_1", header=None).head()

In [14]:
fold_1_result.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,0,15.209765,0.048923,1186.0,14.483480,3,89475.265625,55490.324000,2
1,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,1,14.872161,0.106219,3656.0,13.839700,16,127.850334,17.943619,2
2,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,2,14.732368,0.062658,3314.0,13.559328,27,308.502594,27.946726,2
3,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,3,14.369542,0.079895,3627.0,12.782021,46,119.292923,10.711985,2
4,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,4,14.335183,0.055604,3399.0,12.703944,42,4.917006,0.239674,2


In [15]:
plot_SLIM_fitness_logs('SLIM', n_outer_cv, dataset_name='sustavianfeed')

In [16]:
plot_SLIM_average_fitness('SLIM', n_outer_cv, dataset_name='sustavianfeed')

Except for Fold 6 and 7 that stagnate early on, all folds show a good ability to learn with train and test having close performances. We can conclude that the model is mostly not overfitting. On average, the model look to be stabilyzing so if we were to run more generations we may conclude that the model reach its limits if it doesn't breakthrough from the current level of performance.

In [17]:
plot_SLIM_size_logs('SLIM', n_outer_cv, dataset_name='sustavianfeed')

In [18]:
plot_SLIM_average_size('SLIM', n_outer_cv, dataset_name='sustavianfeed')

There is a clear tendency for the model to grow albeit not exponentially. We observe in several folds a slight tendency for the growth to atagnate or evven decrease towards the end of the iteration (Notabe exceptions to Folds 1 and specially 9 that continue to grow until the end). If we look at the average train/test fitness vs node size we notice that the node count kept growing but (except between generation 7 and 8) it was a slow more incremental growth which was accompanied by a positive evolution of fitness. A situation to look at in further iteration would be what would happen if we ran more generations since we had a considerable leap from generation 19 to 20 and fitness seems to be stabilyzing. If this were to be revelead as a trend we might be looking at eraly signs of bloat.

In [19]:
plot_SLIM_fitness_and_size_logs('SLIM', n_outer_cv, dataset_name='sustavianfeed')

When considering performance vs size on a per-fold basis, we may conclude that the augment in size is justified mostly since it is accompanied by better performance. That isn't true for fold 6 and 7 where the model stopped improving at the third generation but the node count kept growing.

In [20]:
RESULTS = './results/SLIM'
CSV = os.path.join(RESULTS, 'all_folds_grid_search_results.csv')
VERSION_COL = 'param_slim_version'
TEST_SCORE_COL = 'mean_test_score'
DATASET = 'sustavianfeed'
LOG_PATTERN = os.path.join(RESULTS, 'logs_best_params_fold_*')


In [21]:
df_csv = pd.read_csv(CSV)
df_csv['test_rmse'] = -df_csv['mean_test_score']
folds_by_version = (
    df_csv
      .groupby('param_slim_version')['outer_fold']
      .unique()
      .apply(list)
      .to_dict()
)
VERSIONS = list(folds_by_version.keys())

In [22]:
#BOXPLOT: TEST RMSE per Version
df_csv = pd.read_csv(CSV)
df_csv['test_rmse'] = -df_csv[TEST_SCORE_COL]  # convert score to RMSE
fig1 = px.box(
    df_csv, x=VERSION_COL, y='test_rmse', points='all',
    title=f"SLIM – Test RMSE Distribution ({DATASET})"
)
fig1.update_layout(width=700, height=400,
                   xaxis_title="SLIM Version", yaxis_title="Test RMSE")
fig1.show()


The version SLIM * SIG1 appears to be the best for the problem in hand, it has the the lowest bound by far and the distribution of values is closer to it than the upper bound. However, we cannot conclude that it is statistically significant as its upper bound is close to the the median of SLIM * ABS meaning that it is posible than SLIM * ABS may have better performance. Same can be said about SLIM*SIG2 whose lower bound roughly coincides SLIM * SIG1's upper bound. Further statiscal analysis needs to be performed to conclude about the statitiscal significance of this difference. Given this, the first conclusion to be taken is that multiplication provides better results than addition

We shall perform a statistical test considering our null hypothesis as there being no difference between versions and our alternative hypothesis is that the difference differs from zero.

In [23]:
test_versions = ["SLIM*SIG1", "SLIM*SIG2", "SLIM*ABS"]

for v1, v2 in combinations(test_versions, 2):
    s1 = df_csv.loc[df_csv[VERSION_COL] == v1, 'test_rmse']
    s2 = df_csv.loc[df_csv[VERSION_COL] == v2, 'test_rmse']
    # both s1 and s2 must be same length & aligned across folds
    if len(s1) != len(s2):
        raise ValueError(f"Unequal sample sizes: {v1} has {len(s1)} rows, {v2} has {len(s2)}")
    
    stat, p = wilcoxon(s1, s2)
    print(f"Wilcoxon {v1} vs {v2}: stat={stat:.4f}, p‐value={p:.4f}")

Wilcoxon SLIM*SIG1 vs SLIM*SIG2: stat=0.0000, p‐value=0.0020
Wilcoxon SLIM*SIG1 vs SLIM*ABS: stat=4.0000, p‐value=0.0137
Wilcoxon SLIM*SIG2 vs SLIM*ABS: stat=3.0000, p‐value=0.0098


After performing wilcoxon statistical test, we confirm that the differences between models are statistically significant at 95% level of confidence. In other words, at the 5% significance level, SLIM * SIG1 has a significantly lower test-RMSE than both SLIM * SIG2 (p=0.0020) and SLIM * ABS (p=0.0137), and SLIM * SIG2 also outperforms SLIM * ABS (p=0.0098).
Finally, we can reject the hypothesis that their RMSE distributions are the same, and conclude that SLIM * SIG1 is the best of the three.

In [24]:
plot_SLIM_population_fitness_diversity_logs(    
    model_name='SLIM',
    n_folds=n_outer_cv,
    dataset_name='sustavianfeed'
)

Fitness diversity drops almost imeadiatley to close to zero as soon as the model starts iterating. This means that our model is learning too fast, pending heavily towards explotation and neglecting exploration. All solutions are similar meaning the we are facing early convergence. 

In [25]:
plot_SLIM_population_semantic_diversity_logs(
    model_name='SLIM',
    n_folds=n_outer_cv,
    dataset_name='sustavianfeed'
)

Considering semantic diversity, all solution we observed were similar as we notice very low diversity. This lays credence to our previous conclusion that more focus ought to be put in making our model more exploration focused.

Overall, the model shows good performance when compared to node size, few signs of overfitting and bloating not being an immediate concern. Clearly the biggest problem is the lack of semantic and fitness diversity. Although the model has achieved good results with RMSE closing in on zero, the fact that the model fails to explore more of the search space might entail the possibility that better solutions were not avaluated.

Given our analysis conclusion that slim version SLIM * SIG1 is statiscally the versions with better performance, in most cases, for further study we shall focus on improving the model ability to explore the search space. Although bloating is not an immediate concern, in further iterations we shall pay attention to the evolution of performance and bloating as the model looked to stabilize the performance. If we were to run more iterations, we would looking to verify if the model would be able to start a new downward trend without needing big leaps of node count. If not, we might need to adjust p_inflate. Overall, we conclude that our model has solid performance while having a reasonable computational cost. Nevertheless, computational cost may be a impediment going forward as we try to observe the influence of more combinations of parameters.